In [1]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

import folium # map rendering library #!pip install folium
%matplotlib inline

In [2]:
# Data in Wiki has changed; Have to use an older verion to recieve old version of the data
source = requests.get\
    (\
    'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=862527922'\
    ).text

In [3]:
# Extracted Data from Wiki
postal_code=BeautifulSoup(source,'lxml')
df=pd.read_html(str(postal_code.table))[0][1:]
# Dataframe columns: PostalCode, Borough, and Neighborhood
df.columns=pd.read_html(str(postal_code.table))[0].iloc[0]

In [4]:
df.head(15)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


### Data preprocessing

In [5]:
# 1. Only process the cells that have an assigned borough.
idx=df[df['Borough']=="Not assigned"].index
df.drop(idx, axis=0, inplace=True)
df.reset_index()
df.head(15)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


In [6]:
# 2. More than one neighborhood can exist in one postal code area
# Combine into one column
tor=df.groupby(['Postcode', 'Borough'], sort = False).agg( ', '.join).reset_index()
tor.head(15)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
# 3. If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough.
tor.loc[tor['Neighbourhood'] == 'Not assigned','Neighbourhood']=tor['Borough']

In [9]:
# 4. Shape of data frame
tor.shape

(103, 3)